### CDCS_xml

This notebook includes some methods for querying and obtaining results from the ambench.nist.gov website.

In [1]:
import pandas as py
import lxml.etree as et
from cdcs import CDCS
import requests

In [2]:
#Given a keyword to search for, will return a dataframe of seach results
def CDCS_results(keyword):
    curator = CDCS('https://ambench.nist.gov/', username='')
    dfget=curator.query(template='AM-Bench-2018',keyword=keyword)
    return dfget

In [3]:
#Given a dataframe of search results (like the one returned from CDCS_results) and the index of a selected search result,
#will return a neatly formatted version of the xml file (string)
def CDCS_xml(index, df):
    xml=df.iloc[index]['xml_content']
    root=et.fromstring(xml)
    return et.tostring(root, pretty_print=True).decode()

In [4]:
#Given an xml string like the one returned by CDCS_xml, will return a dataframe of download links for the files on the AMBench
#page (includes name, description, comments, etc.)
def CDCS_downloads(xml):
    root=et.fromstring(xml)
    result=[]
    for element in root.iter('downloadURL'):
        entry={}
        for e in element.getparent():
            entry[e.tag]=e.text
        result.append(entry)
    df=py.DataFrame.from_dict(result,orient='columns')
    return df

In [5]:
df=CDCS_results('GS')

In [6]:
df

,id,template,workspace,user_id,title,xml_content,creation_date,last_modification_date,last_change_date,template_title
0,5fb6e80375853f004117ee6a,5fb54ec775853f003743b46f,5fb54c4975853f001635431e,3,AMB2018-02-AMMT-625-TR10-GS.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/2...",2020-11-19T21:47:47.936000Z,2020-11-19T21:47:47.936000Z,2020-11-19T21:51:27.447000Z,AM-Bench-2018
1,5fb6e72475853f003a17eb53,5fb54ec775853f003743b46f,5fb54c4975853f001635431e,3,AMB2018-02-AMMT-625-TR5-GS.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/2...",2020-11-19T21:44:04.965000Z,2020-11-19T21:44:04.965000Z,2020-11-19T21:51:31.807000Z,AM-Bench-2018
2,5fb6e70775853f003917eb9f,5fb54ec775853f003743b46f,5fb54c4975853f001635431e,3,AMB2018-02-AMMT-625-TR3-GS.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/2...",2020-11-19T21:43:35.988000Z,2020-11-19T21:43:35.988000Z,2020-11-19T21:51:32.141000Z,AM-Bench-2018


In [7]:
xml=CDCS_xml(0,df)

In [8]:
print(xml)

<AMBench2018LB xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <BenchMark>
    <Description>Laser powder bed fusion 3D builds and single scan laser traces of nickel-based superalloy IN625 and 15-5 stainless steel</Description>
    <BenchMarkType>
      <BenchMark2>
        <BM2-Description>Laser powder bed fusion of single scan tracks on bare metal substrates</BM2-Description>
        <AMB2-Website>https://www.nist.gov/ambench/amb2018-02-description</AMB2-Website>
        <BM2-ID>
          <BenchMark2-ID>AMB2018-02-AMMT-625-TR10-GS</BenchMark2-ID>
          <TraceID>
            <TraceB>
              <LSLaserPower>179.2</LSLaserPower>
              <LSScanSpeed>800</LSScanSpeed>
              <LSReplications>3</LSReplications>
            </TraceB>
          </TraceID>
          <TrackNumber>10</TrackNumber>
          <AlternateName/>
        </BM2-ID>
      </BenchMark2>
    </BenchMarkType>
    <SampleOwner>
      <Name>MML/642-MW</Name>
    </SampleOwner>
    <Measurement

In [9]:
dfd=CDCS_downloads(xml)

In [10]:
dfd

,name,description,type,downloadURL
0,AMB2018-02-Trace10-195_800_AMMT.tif,The SEM cross sections are mosaics of numerous...,TIFF,https://ambench.nist.gov/rest/blob/download/5f...
1,AMB2018-02-Trace10-195_800_AMMT,The SEM cross sections are mosaics of numerous...,PNG,https://ambench.nist.gov/rest/blob/download/5f...
2,AMB2018-02-625-Trace10-Repolish_EBSD_750x_1805...,NR-processed map with noise reduction,TIFF,https://ambench.nist.gov/rest/blob/download/5f...
3,AMB2018-02-625-Trace10-Repolish_EBSD_750x_1805...,NR-processed map with noise reduction,PNG,https://ambench.nist.gov/rest/blob/download/5f...
4,AMB2018-02-625-Trace10-Repolish_EBSD_750x_1805...,RAW-unprocessed map,TIFF,https://ambench.nist.gov/rest/blob/download/5f...
5,AMB2018-02-625-Trace10-Repolish_EBSD_750x_1805...,RAW-unprocessed map,PNG,https://ambench.nist.gov/rest/blob/download/5f...
6,AMB2018-02-625-Trace10-Repolish_EBSD_750x_1805...,NR (noise reduction) -Euler angles for each gr...,text,https://ambench.nist.gov/rest/blob/download/5f...
7,AMB2018-02-625-Trace10-Repolish_EBSD_750x_1805...,RAW-Euler angles for each grid point,NaN,https://ambench.nist.gov/rest/blob/download/5f...


In [11]:
r=requests.get(dfd['downloadURL'][1])
open('semmp.png','wb').write(r.content)

19471990